In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
data=pd.read_csv("/content/train.csv")
data.head()

,text,sentiment
0,"I`d have responded, if I were going",neutral
1,Sooo SAD I will miss you here in San Diego!!!,negative
2,my boss is bullying me...,negative
3,what interview! leave me alone,negative
4,"Sons of ****, why couldn`t they put them on t...",negative


In [ ]:
data.shape

(27481, 2)

In [ ]:
emotion_mapping = {'positive': 1, 'neutral': 2, 'negative': 0}

In [ ]:
data['sentiment'] = data['sentiment'].map(emotion_mapping)

In [ ]:
data.head()

,text,sentiment
0,"I`d have responded, if I were going",2
1,Sooo SAD I will miss you here in San Diego!!!,0
2,my boss is bullying me...,0
3,what interview! leave me alone,0
4,"Sons of ****, why couldn`t they put them on t...",0


In [ ]:
data['sentiment'].value_counts()

sentiment
2    11118
1     8582
0     7781
Name: count, dtype: int64

In [ ]:
import re
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

In [ ]:
# Download NLTK resources (if not already downloaded)
nltk.download('stopwords')

stopwords_set = set(stopwords.words('english'))
emoji_pattern = re.compile('(?::|;|=)(?:-)?(?:\)|\(|D|P)')

def preprocessing(text):


    # Remove HTML tags
    text = re.sub('<[^>]*>', '', text)

    # Remove URLs
    text = re.sub(r'http\S+', '', text)

    # Extract emojis
    emojis = emoji_pattern.findall(text)

    # Remove non-alphanumeric characters, convert to lowercase, and join emojis
    text = re.sub('[\W+]', ' ', text.lower()) + ' '.join(emojis).replace('-', '')



    # Apply stemming and remove stopwords
    prter = PorterStemmer()
    text = [prter.stem(word) for word in text.split() if word not in stopwords_set]

    return " ".join(text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
preprocessing('this is im my tags <h1> :) <p>helo world<p> <div> <div> </h2>')

'im tag helo world :)'

In [ ]:
print(data['text'].isnull().sum())

1


In [ ]:
data['text'] = data['text'].fillna('')

In [ ]:
data['text']

0                      I`d have responded, if I were going
1            Sooo SAD I will miss you here in San Diego!!!
2                                my boss is bullying me...
3                           what interview! leave me alone
4         Sons of ****, why couldn`t they put them on t...
                               ...                        
27476     wish we could come see u on Denver  husband l...
27477     I`ve wondered about rake to.  The client has ...
27478     Yay good for both of you. Enjoy the break - y...
27479                           But it was worth it  ****.
27480       All this flirting going on - The ATG smiles...
Name: text, Length: 27481, dtype: object

In [ ]:
data['text'] = data['text'].apply(lambda x: preprocessing(x))

In [ ]:
data['text']

0                                               respond go
1                                  sooo sad miss san diego
2                                               boss bulli
3                                      interview leav alon
4                            son put releas alreadi bought
                               ...                        
27476    wish could come see u denver husband lost job ...
27477    wonder rake client made clear net forc dev lea...
27478    yay good enjoy break probabl need hectic weeke...
27479                                                worth
27480                           flirt go atg smile yay hug
Name: text, Length: 27481, dtype: object

In [ ]:
pip install keras

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense,Dropout
from keras.utils import to_categorical
from keras.layers import *

In [ ]:
vocab_size = len(set(" ".join(data['text']).split()))
vocab_size

20266

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
# TF-IDF Vectorization
tfidf=TfidfVectorizer(strip_accents=None,lowercase=False,preprocessor=None,use_idf=True,norm='l2',smooth_idf=True,max_features=1000)
x = tfidf.fit_transform(data.text).toarray()
y = data.sentiment.values
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
x_train.shape

(21984, 1000)

In [ ]:
x_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
# Reshape the input data for LSTM
# LSTM expects input in the shape of (samples, time steps, features)
# In this case, samples would be the number of documents, time steps would be 1 (since each document is treated independently), and features would be the TF-IDF values
x_train_lstm = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
x_test_lstm = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))

In [ ]:
from keras.layers import Input

# Define input layer explicitly
input_layer = Input(shape=(1, x_train.shape[1]))

# Define LSTM model
model = Sequential()
model.add(input_layer)
model.add(LSTM(units=128))
model.add(Dense(units=3, activation='softmax'))  # 3 output classes: positive, negative, neutral

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

print(model.summary())


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 128)               578048    
                                                                 
 dense_1 (Dense)             (None, 3)                 387       
                                                                 
Total params: 578435 (2.21 MB)
Trainable params: 578435 (2.21 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
### Finally Training
# Train the model
model.fit(x_train_lstm,y_train,validation_data=(x_test_lstm,y_test),batch_size=64,epochs=10)

Epoch 1/10
344/344 [==============================] - 14s 26ms/step - loss: 0.9157 - accuracy: 0.5757 - val_loss: 0.7623 - val_accuracy: 0.6853
Epoch 2/10
344/344 [==============================] - 5s 16ms/step - loss: 0.7170 - accuracy: 0.7066 - val_loss: 0.7363 - val_accuracy: 0.7009
Epoch 3/10
344/344 [==============================] - 4s 13ms/step - loss: 0.6927 - accuracy: 0.7164 - val_loss: 0.7344 - val_accuracy: 0.7031
Epoch 4/10
344/344 [==============================] - 4s 11ms/step - loss: 0.6817 - accuracy: 0.7198 - val_loss: 0.7333 - val_accuracy: 0.7029
Epoch 5/10
344/344 [==============================] - 4s 11ms/step - loss: 0.6752 - accuracy: 0.7238 - val_loss: 0.7348 - val_accuracy: 0.7040
Epoch 6/10
344/344 [==============================] - 4s 12ms/step - loss: 0.6695 - accuracy: 0.7262 - val_loss: 0.7363 - val_accuracy: 0.7018
Epoch 7/10
344/344 [==============================] - 4s 12ms/step - loss: 0.6646 - accuracy: 0.7275 - val_loss: 0.7364 - val_accuracy: 0.702

In [ ]:
# Evaluate the model
score = model.evaluate(x_test_lstm, y_test)
print('Test Accuracy:', score[1])

172/172 [==============================] - 1s 3ms/step - loss: 0.7344 - accuracy: 0.7024
Test Accuracy: 0.7023831009864807


In [ ]:
import pickle
pickle.dump(model,open('model.pkl','wb'))
pickle.dump(tfidf,open('tfidf.pkl','wb'))

In [ ]:
def prediction(comment):
    preprocessed_comment = preprocessing(comment)
    print("Preprocessed Comment:", preprocessed_comment)
    comment_list = [preprocessed_comment]
    comment_vector = tfidf.transform(comment_list).toarray()
    # print("TF-IDF Vector:", comment_vector)

    # Reshape TF-IDF vector for LSTM input
    comment_vector_lstm = np.reshape(comment_vector, (comment_vector.shape[0], 1, comment_vector.shape[1]))

    decision_scores = model.predict(comment_vector_lstm)
    print("Raw Decision Scores:", decision_scores)
    predicted_class = np.argmax(decision_scores)

    if predicted_class == 1:
        print("Positive comment")
    elif predicted_class == 0:
        print("Negative comment")
    else:
        print("Neutral comment")

prediction('Im so disappointed with the quality of the product I received..')


Preprocessed Comment: im disappoint qualiti product receiv
1/1 [==============================] - 0s 20ms/step
Raw Decision Scores: [[0.9816682  0.00323635 0.01509549]]
Negative comment


In [ ]:
model.save('sentiment_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
prediction("Todays lecture covered various topics related to modern art")

Preprocessed Comment: today lectur cover variou topic relat modern art
TF-IDF Vector:   (0, 885)	0.5621660998227641
  (0, 876)	0.3011756360947029
  (0, 204)	0.5314382487283027
  (0, 65)	0.5575265914928254


InvalidArgumentError: {{function_node __wrapped__SerializeManySparse_device_/job:localhost/replica:0/task:0/device:CPU:0}} indices[1] = [0,876] is out of order. Many sparse ops require sorted indices.
    Use `tf.sparse.reorder` to create a correctly ordered copy.

 [Op:SerializeManySparse] name: 